In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


In [2]:
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    tone_list = []
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints
        

In [3]:

piece = importScore('https://crimproject.org/mei/CRIM_Model_0015.mei')


Successfully imported https://crimproject.org/mei/CRIM_Model_0015.mei


In [4]:
df = piece.getMelodicEntries(n=5)
piece.detailIndex(df, offset=True)

Superior        Contratenor             Tenor  \
Measure Beat Offset                                                          
1       1.0  0.0       5, -2, 2, 2, 2                NaN               NaN   
3       1.0  16.0                 NaN     4, -2, 2, 2, 2               NaN   
5       4.0  38.0    -3, 2, -2, -2, 2                NaN               NaN   
7       2.0  50.0                 NaN   -3, 2, -2, -2, 2               NaN   
        4.0  54.0                 NaN                NaN               NaN   
...                               ...                ...               ...   
159     2.0  1274.0               NaN  3, -2, -3, -2, -2               NaN   
160     4.0  1286.0               NaN                NaN               NaN   
163     2.0  1306.0               NaN                NaN               NaN   
        4.0  1310.0               NaN   3, -2, -2, -2, 2               NaN   
164     2.0  1314.0               NaN                NaN  2, -2, -2, -2, 2   

                                Bassus  
Measure Beat Offset                     
1       1.0  0.0                   NaN  
3       1.0  16.0                  NaN  
5       4.0  38.0                  NaN  
7       2.0  50.0                  NaN  
        4.0  54.0     -3, 2, -2, -2, 2  
...                                ...  
159     2.0  1274.0                NaN  
160     4.0  1286.0  3, -2, -2, -2, -2  
163     2.0  1306.0   3, -2, -2, -2, 2  
        4.0  1310.0                NaN  
164     2.0  1314.0                NaN  

[104 rows x 4 columns]

In [6]:
piece = importScore('https://crimproject.org/mei/CRIM_Model_0015.mei')

nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=5)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=5)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()


#  now build up metata for each group of similar soggetti
# stack the similar soggetti
# extract list of voices
# extract list of starting offsets
# get time ints
# extract starting pitches for each
# get entry ints
# make a 'row' with offsets, voices, time ints and entry ints
# add that row to DF

points = pd.DataFrame()
for matches in full_list_of_matches["match"]:
# for matches in test:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
    voice_list = entry_array['voice'].to_list()
    offset_list = entry_array.index.to_list()

    time_ints = numpy.diff(offset_list).tolist()
    tone_coordinates =  list(zip(offset_list, voice_list))
    mel_ints = find_entry_int_distance(tone_coordinates, piece)
    first_offset = offset_list[0]
#     first_note_list = nr[offset_list]
    meas_beat = det[det.index.get_level_values('Offset').isin(offset_list)]
    mb2 = meas_beat.reset_index()
    mb2['mb'] = mb2["Measure"].astype(str)+"."+mb2["Beat"].astype(str)
    meas_beat_list = mb2['mb'].to_list()

    temp = {'First_Offset': first_offset, 
            'Offsets': offset_list, 
            'Measures_Beats': meas_beat_list,
            "Soggetti": matches,
            'Voices': voice_list, 
            'Time_Entry_Intervals': time_ints, 
            'Melodic_Entry_Intervals': mel_ints}
    temp
    points = points.append(temp, ignore_index=True).sort_values("First_Offset")
points_no_singles = points[points['Offsets'].apply(len) > 1]
points_no_singles.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)


points_no_singles.head()


Memoized piece detected.


/var/folders/k5/k4j_550s2yq5776vgmw6r_xr0000gp/T/ipykernel_21615/585453238.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_no_singles.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)


,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices
0,0.0,"[1.1.0, 3.1.0, 10.1.0, 12.2.0]","[P-4, P-5, P-4]","[0.0, 16.0, 72.0, 90.0]","[5, -2, 2, 2, 2, 4, -2, 2, 2, 2]","[16.0, 56.0, 18.0]","[Superior, Contratenor, Tenor, Bassus]"
2,38.0,"[5.4.0, 7.2.0, 7.4.0, 10.4.0, 14.4.0, 16.4.0, ...","[P-5, P-4, P8, P-8, P5, P4, P-8]","[38.0, 50.0, 54.0, 78.0, 110.0, 126.0, 138.0, ...","[-3, 2, -2, -2, 2, -3, 2, -2, -2, 1]","[12.0, 4.0, 24.0, 32.0, 16.0, 12.0, 8.0]","[Superior, Contratenor, Bassus, Superior, Teno..."
5,174.0,"[22.4.0, 24.2.0, 26.4.0, 28.2.0, 154.2.0, 154....","[P5, P-12, P5, P8, P-8, M-2, P-4, P12, P-12]","[174.0, 186.0, 206.0, 218.0, 1234.0, 1238.0, 1...","[3, -2, -2, -2, -3, 3, -2, -2, -2, -2]","[12.0, 20.0, 12.0, 1016.0, 4.0, 16.0, 4.0, 12....","[Contratenor, Superior, Bassus, Tenor, Superio..."
10,266.0,"[34.2.0, 38.2.0, 40.2.0]","[P1, P-5]","[266.0, 298.0, 314.0]","[2, -3, 5, 1, -5]","[32.0, 16.0]","[Bassus, Tenor, Bassus]"
11,274.0,"[35.2.0, 36.2.0, 41.2.0]","[P-5, P1]","[274.0, 282.0, 322.0]","[2, -3, 5, 1, -2]","[8.0, 40.0]","[Superior, Contratenor, Tenor]"


In [557]:
_list = [0.0,
 16.0,
 38.0,
 50.0,]
mel_n = piece.getMelodic(kind="d", compound=True)
ngrams = piece.getNgrams(df=mel_n, n=5)
ngrams.head(20)
ngrams.iloc[_list, :]
# rows = a_dataframe.iloc[list_of_indexes, :]

,Superior,Contratenor,Tenor,Bassus
4.0,"5, -2, 2, 2, 2",NaN,NaN,NaN
42.0,"2, -2, -2, 2, 2",NaN,NaN,NaN
77.0,NaN,"-2, -2, -2, 2, -8",NaN,"-2, 2, 2, 2, -2"
94.0,"2, 2, 2, 2, 2","8, -2, 2, -2, -2","-2, -3, 3, -2, 2",NaN


In [527]:
related_entry_list

,,pattern
798.0,Bassus,"5, 1, -3, 2, 2"
826.0,Contratenor,"5, 1, -3, 2, 2"
838.0,Superior,"5, 1, -3, 2, 2"
850.0,Tenor,"4, 1, -3, 2, 2"
866.0,Bassus,"4, 1, -3, 2, 2"


In [523]:
temp

{'First_Offset': 798.0,
 'Offsets': [798.0, 826.0, 838.0, 850.0, 866.0],
 'Measures_Beats': ['99.4.0', '103.2.0', '104.4.0', '106.2.0', '108.2.0'],
 'Soggetti': ['5, 1, -3, 2, 2', '4, 1, -3, 2, 2'],
 'Voices': ['Bassus', 'Contratenor', 'Superior', 'Tenor', 'Bassus'],
 'Time_Entry_Intervals': [28.0, 12.0, 12.0, 16.0],
 'Melodic_Entry_Intervals': ['P5', 'P4', 'P-4', 'P-8']}

In [7]:
# Durations of Notes in Ngrams
nr = piece.getNoteRest()
dur = piece.getDuration(df=nr)
ng = piece.getNgrams(df=dur, n=5)
ng

,Superior,Contratenor,Tenor,Bassus
0.0,"(4.0, 6.0, 2.0, 2.0, 2.0)","(16.0, 4.0, 6.0, 2.0, 2.0)","(72.0, 4.0, 6.0, 2.0, 2.0)","(54.0, 2.0, 2.0, 4.0, 2.0)"
4.0,"(6.0, 2.0, 2.0, 2.0, 3.0)",NaN,NaN,NaN
10.0,"(2.0, 2.0, 2.0, 3.0, 0.5)",NaN,NaN,NaN
12.0,"(2.0, 2.0, 3.0, 0.5, 0.5)",NaN,NaN,NaN
14.0,"(2.0, 3.0, 0.5, 0.5, 2.0)",NaN,NaN,NaN
...,...,...,...,...
1316.0,NaN,"(1.0, 1.0, 2.0, 2.0, 2.0)","(3.0, 1.0, 2.0, 2.0, 4.0)","(2.0, 1.0, 1.0, 1.0, 1.0)"
1317.0,NaN,"(1.0, 2.0, 2.0, 2.0, 4.0)",NaN,NaN
1318.0,NaN,"(2.0, 2.0, 2.0, 4.0, 16.0)",NaN,"(1.0, 1.0, 1.0, 1.0, 2.0)"
1319.0,NaN,NaN,"(1.0, 2.0, 2.0, 4.0, 16.0)","(1.0, 1.0, 1.0, 2.0, 4.0)"


In [506]:
import itertools

In [507]:
data = [4, 8, 4, 20, 4]

result = itertools.takewhile(lambda x: x<10, data)
for each in result:
    print(each)

4
8
4


In [515]:
data = [4, 8, 4, 20, 4]

result = itertools.dropwhile(lambda x: x<10, data)
for each in result:
    print(each)

20
4


In [ ]:


def before_and_after(predicate, it):
    """ Variant of takewhile() that allows complete
        access to the remainder of the iterator.

        >>> it = iter('ABCdEfGhI')
        >>> all_upper, remainder = before_and_after(str.isupper, it)
        >>> ''.join(all_upper)
        'ABC'
        >>> ''.join(remainder)     # takewhile() would lose the 'd'
        'dEfGhI'

        Note that the first iterator must be fully
        consumed before the second iterator can
        generate valid results.
    """
    it = iter(it)
    transition = []
    def true_iterator():
        for elem in it:
            if predicate(elem):
                yield elem
            else:
                transition.append(elem)
                return
    def remainder_iterator():
        yield from transition
        yield from it
    return true_iterator(), remainder_iterator()